In [1]:
import mysql.connector

У системного администратора есть список оборудования и список сотрудников. Необходимо хранить что у кого находится.

## Create database

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="mandarinka"
)

In [3]:
mycursor = mydb.cursor()

**create**

In [7]:
mycursor.execute("CREATE DATABASE techdata;")

In [9]:
mycursor.execute("ALTER DATABASE techdata CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")

In [10]:
mydb.commit()

**check**

In [11]:
mycursor.execute("SHOW DATABASES")
mycursor.fetchall()

[('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sys',),
 ('techdata',)]

## Generate data

In [129]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="mandarinka",
  database="techdata"
)
mycursor = mydb.cursor()

In [13]:
mycursor.execute('CREATE TABLE hardware (\
id INT KEY AUTO_INCREMENT, id_char INT, out_date DATE, bought DATE, owner INT)')
mydb.commit()

In [14]:
mycursor.execute('CREATE TABLE type (\
id INT KEY AUTO_INCREMENT, htype TEXT, producer TEXT, model TEXT, charact TEXT, year INT)')
mydb.commit()

In [15]:
mycursor.execute('CREATE TABLE workers (\
id INT KEY AUTO_INCREMENT, name TEXT, department TEXT, position TEXT, birth DATE, office INT)')
mydb.commit()

In [16]:
mycursor.execute('SHOW TABLES')
mycursor.fetchall()

[('hardware',), ('type',), ('workers',)]

In [18]:
import time
import datetime
import random

def date_to_timestamp(d) :
    return int(time.mktime(d.timetuple()))

def randomDate(start, end):
    """Get a random date between two dates"""

    stime = date_to_timestamp(start)
    etime = date_to_timestamp(end)

    ptime = stime + random.random() * (etime - stime)

    return datetime.date.fromtimestamp(ptime)

def names():
    with open('names.txt','r', encoding='utf-8') as f:
        for line in f:
            yield line.strip()

deps = ['машинный перевод', 'отдел кадров', 'веб-дизайн', 'бухгалтерия', 'реклама']
positions = {'машинный перевод':['старший разработчик', 'младший разработчик', 'стажер'],
             'отдел кадров':['рекрутер','старший специалист','аналитик'],
             'веб-дизайн':['дизайнер','руководитель группы','разработчик'], 
             'бухгалтерия':['старший бухгалтер','бухгалтер','аудитор'],
             'реклама':['старший аналитик','аналитик','оператор call-центра',
                        'специалист по ТВ', 'специалист по соцсетям']}
off = {'машинный перевод':[1,2,3,4,5],
             'отдел кадров':[6,7,8,9],
             'веб-дизайн':[10,11], 
             'бухгалтерия':[12,13],
             'реклама':[14,15]}

def department_position():
    global deps, position
    dep = random.choice(deps)
    pos = random.choice(positions[dep])
    offi = random.choice(off[dep])
    return dep, pos, offi

def fill_workers():
    data = []
    for name in names():
        dep, pos, offi = department_position()
        birth = randomDate(datetime.date(year=1930, month=1, day=1), datetime.date(year=2000, month=1, day=1))
        data.append((name, dep, pos, str(birth), offi))
    #return data
    mycursor.executemany('INSERT INTO workers (name, department, position, birth, office) \
    VALUES (%s, %s, %s, %s, %s)', data)
    mydb.commit()

In [19]:
fill_workers()

In [ ]:
mycursor.execute('SELECT * FROM workers')
print (mycursor.fetchall())

In [21]:
with open ('hardware.txt','r') as f:
    data = [tuple(line.strip().split(',')+[]) for line in f.readlines()]

In [23]:
def fill_types():
    with open ('hardware.txt','r') as f:
        data = [tuple(line.strip().split(',')+['характеристика', random.randint(2016, 2018)]) for line in f.readlines()]
    mycursor.executemany('INSERT INTO type (htype, producer, model, charact, year) \
    VALUES (%s, %s, %s, %s, %s)', data)
    mydb.commit()

In [24]:
fill_types()

In [27]:
mycursor.execute('SELECT * FROM type')
mycursor.fetchall()

[(1, 'ноутбук', 'Lenovo', 'Yoga 370', 'характеристика', 2018),
 (2, 'ноутбук', 'Lenovo', 'X1 Extreme', 'характеристика', 2018),
 (3, 'ноутбук', 'Lenovo', 'X1 Carbon (6th)', 'характеристика', 2018),
 (4, 'ноутбук', 'Lenovo', 'X1 Carbon (5th Gen)', 'характеристика', 2018),
 (5, 'ноутбук', 'Lenovo', 'X1 Yoga (3rd Gen)', 'характеристика', 2016),
 (6, 'ноутбук', 'Lenovo', 'X1 Yoga (2nd Gen)', 'характеристика', 2017),
 (7, 'ноутбук', 'Lenovo', 'X380 Yoga', 'характеристика', 2018),
 (8, 'ноутбук', 'Lenovo', 'X280', 'характеристика', 2016),
 (9, 'ноутбук', 'Lenovo', 'X270', 'характеристика', 2016),
 (10, 'ноутбук', 'Lenovo', 'T580', 'характеристика', 2018),
 (11, 'ноутбук', 'Lenovo', 'T570', 'характеристика', 2018),
 (12, 'ноутбук', 'Lenovo', 'T480', 'характеристика', 2018),
 (13, 'ноутбук', 'Lenovo', 'T470p', 'характеристика', 2017),
 (14, 'ноутбук', 'Lenovo', 'T470', 'характеристика', 2018),
 (15, 'ноутбук', 'Lenovo', 'T480s', 'характеристика', 2016),
 (16, 'ноутбук', 'Lenovo', 'P1', 'характ

In [35]:
def fill_hardware():
    data = []
    for _ in range(600):
        #id_char INT, out_date DATE, bought DATE, owner INT
        id_char = random.randint(1,34)
        out_date = randomDate(datetime.date(year=2016, month=1, day=1), datetime.date(year=2018, month=11, day=18))
        bought = randomDate(datetime.date(year=2016, month=1, day=1), out_date)
        owner = random.randint(1,300)
        data.append((id_char, out_date, bought, owner))
        
    mycursor.executemany('INSERT INTO hardware (id_char, out_date, bought, owner) \
    VALUES (%s, %s, %s, %s)', data)
    mydb.commit()

In [36]:
fill_hardware()

In [38]:
mycursor.execute('SELECT * FROM hardware WHERE owner=5')
mycursor.fetchall()

[(8, 31, datetime.date(2016, 6, 27), datetime.date(2016, 4, 28), 5),
 (26, 11, datetime.date(2017, 5, 19), datetime.date(2017, 1, 5), 5),
 (281, 3, datetime.date(2016, 11, 22), datetime.date(2016, 4, 23), 5),
 (398, 11, datetime.date(2018, 4, 23), datetime.date(2017, 7, 25), 5),
 (527, 3, datetime.date(2016, 11, 25), datetime.date(2016, 9, 20), 5)]

In [80]:
mycursor.execute('SELECT * FROM hardware INNER JOIN type ON hardware.id_char = type.id INNER JOIN workers ON workers.id=hardware.owner')
#len(mycursor.fetchall())

In [181]:
%%timeit
mycursor.execute('SELECT type.htype, type.producer, type.model, hardware.out_date, workers.name \
FROM type JOIN hardware ON type.id = hardware.id_char JOIN workers ON workers.id = hardware.owner WHERE hardware.out_date > "2018-01-01"')#' INNER JOIN workers ON workers.id = hardware.owner \
a = mycursor.fetchall()

5.44 ms ± 590 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [218]:
mycursor.execute('SELECT type.htype, type.producer, type.model, hardware.out_date, workers.name \
FROM type JOIN hardware ON type.id = hardware.id_char AND hardware.out_date > "2018-01-01" JOIN workers ON workers.id = hardware.owner')
a = mycursor.fetchall()

In [ ]:
a

In [185]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="mandarinka",
  database="techdata"
)
mycursor = mydb.cursor()

In [183]:
%time mycursor.execute('SELECT type.htype, type.producer, type.model FROM type')#' INNER JOIN workers ON workers.id = hardware.owner \
len(mycursor.fetchall())

CPU times: user 1.79 ms, sys: 257 µs, total: 2.05 ms
Wall time: 104 ms


In [193]:
#%%time 
mycursor.execute('SELECT * FROM workers WHERE name LIKE "%тим%"')#' INNER JOIN workers ON workers.id = hardware.owner \
mycursor.fetchall()

[(3,
  'Харькова Валерия Тимуровна',
  'машинный перевод',
  'стажер',
  datetime.date(1968, 4, 25),
  4),
 (17,
  'Тимошкин Карл Онисимович',
  'реклама',
  'специалист по соцсетям',
  datetime.date(1999, 5, 5),
  14),
 (76,
  'Кувардин Андриян Тимурович',
  'машинный перевод',
  'старший разработчик',
  datetime.date(1988, 4, 27),
  4),
 (102,
  'Шигаев Тимур Матвеевич',
  'бухгалтерия',
  'старший бухгалтер',
  datetime.date(1956, 7, 2),
  12),
 (175,
  'Полищука Марианна Тимуровна',
  'отдел кадров',
  'рекрутер',
  datetime.date(1945, 2, 16),
  7),
 (199,
  'Нилов Тимур Андреевич',
  'машинный перевод',
  'старший разработчик',
  datetime.date(1987, 2, 10),
  4),
 (268,
  'Кацен Тимур Аполлинариевич',
  'отдел кадров',
  'аналитик',
  datetime.date(1989, 5, 19),
  7),
 (276,
  'Игнаткович Тимофей Сидорович',
  'отдел кадров',
  'рекрутер',
  datetime.date(1984, 7, 1),
  8)]

In [262]:
class DB:
    
    def __init__(self):
        self.db = mysql.connector.connect(
          host="localhost",
          user="root",
          passwd="mandarinka",
          database="techdata"
        )
        self.cur = self.db.cursor()
        
    def create_worker(self, name, department, position, birth, office):
        self.cur.execute('INSERT INTO workers (name, department, position, birth, office) \
    VALUES (%s, %s, %s, %s, %s)', [(name, department, position, birth, office)])
        self.db.commit()
    def update_worker(self, wid, name, department, position, birth, office):
        self.cur.execute('UPDATE workers SET name = %s, department = %s, position = %s, birth = %s, office = %s WHERE id = %s', 
                         [(name, department, position, birth, office, wid)])
        self.db.commit()
    def remove_worker(self, wid):
        self.cur.execute('DELETE * FROM workers WHERE id = %s', wid)
        self.cur.execute('DELETE * FROM hardware WHERE owner = %s', wid)
        self.db.commit()
        
    def create_type(self, htype, producer, model, charact, year):
        self.cur.execute('INSERT INTO type (htype, producer, model, charact, year) \
    VALUES (%s, %s, %s, %s, %s)', [(htype, producer, model, charact, year)])
        self.db.commit()        
    def update_type(self, tid, name, department, position, birth, office):
        self.cur.execute('UPDATE type SET htype = %s, producer = %s, model = %s, charact = %s, year = %s WHERE id = %s', 
                         [(htype, producer, model, charact, year, tid)])
        self.db.commit() 
    def remove_type(self, wid):
        self.cur.execute('DELETE * FROM type WHERE id = %s', wid)
        self.cur.execute('DELETE * FROM hardware WHERE id_char = %s', wid)
        self.db.commit()
        
    def create_hardware(self, id_char, out_date, bought, owner):
        self.cur.execute('INSERT INTO hardware (id_char, out_date, bought, owner) \
    VALUES (%s, %s, %s, %s)', [(id_char, out_date, bought, owner)])
        self.db.commit()
    def update_hardware(self, tid, id_char, out_date, bought, owner):
        self.cur.execute('UPDATE hardware SET id_char = %s, out_date = %s, bought = %s, owner = %s WHERE id = %s', 
                         [(id_char, out_date, bought, owner, tid)])
        self.db.commit()
    def remove_hardware(self, wid):
        self.cur.execute('DELETE * FROM hardware WHERE id = %s', wid)
        self.db.commit()
        
    def search(self, args):
        query = []
        check = []
        columns = []
        if 'use_htype' in args and args['use_htype'] == 'True':
            if args['htype'] != '': 
                query.append('type.htype LIKE \"%{}%\"'.format(args['htype']))
            check.append('type.htype')
            columns.append('<b>Тип устройства</b>')
        if 'use_producer' in args and args['use_producer'] == 'True':
            if args['htype'] != '': 
                query.append('type.producer LIKE \"%{}%\"'.format(args['producer']))
            check.append('type.producer')
            columns.append('<b>Производитель</b>')
        if 'use_model' in args and args['use_model'] == 'True':
            if args['model'] != '': 
                query.append('type.model LIKE \"%{}%\"'.format(args['model']))
            check.append('type.model')
            columns.append('<b>Модель</b>')
        if 'use_out_date' in args and args['use_out_date'] == 'True':
            if args['out_date'] != '':
                query.append('hardware.out_date = \"{}\"'.format(args['out_date']))
            check.append('hardware.out_date')
            columns.append('<b>Дата выдачи</b>')
        if 'use_name' in args and args['use_name'] == 'True':
            if args['name'] != '': 
                query.append('workers.name LIKE \"%{}%\"'.format(args['name']))
            check.append('workers.name')
            columns.append('<b>Имя</b>')
        if 'use_department' in args and args['use_department'] == 'True':
            if args['department'] != '': 
                query.append('workers.department LIKE \"%{}%\"'.format(args['department']))
            check.append('workers.department')
            columns.append('<b>Департамент</b>')
        if 'use_position' in args and args['use_position'] == 'True':
            if args['position'] != '': 
                query.append('workers.position LIKE \"%{}%\"'.format(args['position']))
            check.append('workers.position')
            columns.append('<b>Должность</b>')
        
        if 'use_office' in args and args['use_office'] == 'True':
            if args['office'] != '': 
                query.append('workers.office={}'.format(args['office']))
            check.append('workers.office')
            columns.append('<b>Кабинет</b>')
            
        self.cur.execute('SELECT DISTINCT {} FROM type \
        JOIN hardware ON type.id = hardware.id_char \
        JOIN workers ON workers.id = hardware.owner \
        WHERE {}'.format(', '.join(check), ' AND '.join(query)))
        data = [tuple(columns)]+self.cur.fetchall()
        return data
    
    def search_workers(self, args):
        pass
    
    def search_type(self, args):
        pass
    
    def search_hard(self, args):
        pass
    
    def prettify_search(self, data):
        return '<table><tr><td>{} </td></tr></table>'.format(
            '</td></tr><tr><td>'.join(
                ['</td><td>'.join(str(x) for x in i) for i in data]
            ))

In [263]:
db = DB()

In [266]:
query = {'use_position':'True','position':'разработчик'}#, 
         #'use_htype':'True', 'htype':'', 
         #'use_out_date':'True', 'out_date':'',
         #'use_name':'True', 'name':'А'}

In [267]:
#db.prettify_search(db.search(query))
db.search(query)

[('младший разработчик',), ('старший разработчик',), ('разработчик',)]